# Machine Learning Lab for Tutorial 3

## Question 4

##### *Bonus question - for bonus marks*. We will now look at a more challenging text-based classification problem, namely to classify a page from a Harry Potter book into which of the seven books the page was taken from. The books can be found in the zip file hp books.zip and are text files where each page of a given book is a line in the text file. Note, all punctuation and capital lett

### Question 4 a

###### Train an NB model using 80% of the data to train and the remaining 20% as test data. Use Laplace smoothing for your model. Report a confusion matrix of your results.

In [99]:
import numpy as np
import math

In [100]:
#store the data in a matrix
files = ['hp_books\\HP1.txt', 
         'hp_books\\HP2.txt', 
         'hp_books\\HP3.txt', 
         'hp_books\\HP4.txt', 
         'hp_books\\HP5.txt', 
         'hp_books\\HP6.txt', 
         'hp_books\\HP7.txt']

data = list()

for file in files:
    f = open(file)
    line = f.readline()
    
    while(line):
        data.append(line.rstrip('\n') + file[11])
        line = f.readline()


In [101]:
#select random 80% rows from data for training data
train_data = np.random.choice(data, int(0.8*len(data)), replace=False)

#the remaining 20% is for testing the model
test_data = np.array([line for line in data if line not in train_data])

#print('Train data:\n')
#print(train_data)

#print('\nTest data:\n')
#print(test_data)

In [102]:
#split the traing data into x_train_data and y_train_label
x_train_data = [line.split(' ')[:-1] for line in train_data]
y_train_label = [line.split(' ')[-1] for line in train_data]
y_train_label = np.array(y_train_label).astype(np.int32)

#print('x Train data:\n')
#print(x_train_data)

#print('\ny Train label:\n')
#print(y_train_label)

In [103]:
#selects all the features of every row in the training data
features = list()
for vec in x_train_data:
    for word in vec:
        if word not in features and word != '':
            features.append(word)
#print('All the Features of every data_point\n')
#print(features)

In [104]:
#Prior Probability function
def prior_prob(x):
    number_of_x = 0
    for label in y_train_label:
        if label == x:
            number_of_x += 1
    return number_of_x/len(y_train_label)

In [115]:
#Conditional probability of x given that is class c
def con_prob(x, c):
    count = 0
    number_of_x_and_c = 0
    number_of_c = 0
    
    for vec in x_train_data:
        if y_train_label[count] == c:
            if x in vec:
                number_of_x_and_c += 1
            number_of_c += 1
        count += 1
    return number_of_x_and_c / number_of_c


#Constructing Probability Table
#prob_table stores ['word', con_prob(word, 1), con_prob(word, 2), ..., con_prob(word, 7)]
prob_table = list()
for word in features:
    prob_table.append([word, 
                        con_prob(word, 1), 
                        con_prob(word, 2),
                        con_prob(word, 3),
                        con_prob(word, 4),
                        con_prob(word, 5),
                        con_prob(word, 6),
                        con_prob(word, 7)])
    #print('I am still working.....')

#print(prob_table)

### Question 4 b

###### Adapt your code to use 80% of the data to train, 10% of the data as validation data and the remaining 10% as test data. Train separate NB classifiers using the values {1 × 10−1 , 1 × 10−2 , 1 × 10−3 , 1 × 10−4 , 1×10−5 , 1×10−6} to smooth the table of likelihoods. Train each model using the training data, and track its performance on the validation data. Which model gave the best accuracy on validation data? Does the choice of smoothing value have a big impact on the performance of the model?


In [116]:
#new way for smoothing the data
#inputs: probability table, hyperparameter
def updated_prob_table(param):
    for vec in prob_table:
        for i in range(1, 8):
            vec[i] += param
    

#Training the NB Model
#input : array data_point
def Predicted_class(data_point, param):
    
    d = dict()
    class_labels = np.arange(1, 8)
    
    #likelihood probabilies
    for label in class_labels:
        d[label] = 0
    
    #word doesn't exist in features
    for word in data_point:
        if word not in features:
            features.append(word)
            l = [word]
            for label in d:
                temp = param
                l.append(temp)
            updated_prob_table(param)
            prob_table.append(l)
    
    #computing the probabilities
    for word in features:
        
        i = features.index(word)
        if word in data_point:
            for label in d:
                
                if prob(prob_table[i][label]) == 0:
                    
                    prob_table = updated_prob_table(prob_table, param)
                    d[label] += math.log(prob(prob_table[i][label]))
                    
                else:
                    d[label] += math.log(prob(prob_table[i][label]))
        
        else:
            for label in d:
                
                if prob(prob_table[i][label][0]) == 1:
                    
                    prob_table = updated_prob_table(prob_table, param)
                    d[label] += math.log(1 - prob(prob_table[i][label]))
                    
                    
                else:
                    d[label] += math.log(1 - prob(prob_table[i][label]))
                
                
    denominator = 0
    
    
    #store the prior probabilities
    prior_probabilities = list([1])
        
    
    for label in d:
        prior_probabilities.append(prior_prob(label))
        denominator += math.exp(d[label]) * prior_probabilites[label]
    
    
    for label in d:
        if denominator == 0: denominator = 1
        d[label] = math.exp(d[label] + math.log(prior_probabilities[label]) - math.log(denominator))
    
    
    return [label for label in d.keys() if d[label] == max(d.values())][0]
    

In [117]:
#spliting the test data into validation and testing data. 10% validation, 10% testing from the 20%
validate_data = np.random.choice(test_data, int(0.5*len(test_data)), replace=False)

#the remaining 10% is for testing the model
test_data = np.array([line for line in test_data if line not in validate_data])

In [118]:
#Validating the models of NB
#NB hyperparameters for the models
NB_Model = pow(10, -1)


#validation data
x_validate_data = [line.split(' ')[:-1] for line in validate_data]
y_validate_label = [line.split(' ')[-1] for line in validate_data]
#print(y_test_label)

    
y_Predicted_label = list()
count = 0
for vec in x_validate_data:
    y_Predicted_label.append(Predicted_class(vec, NB_Model))
    count += 1
    print(count)
#print(y_Predicted_label)


#computing confusion matrix
class_labels = [1, 2, 3, 4, 5, 6, 7]
confusion_matrix = list()

count = 0

for label1 in class_labels:
    l = list()
    for label2 in class_labels:
        for i in range(len(y_validate_label)):
            if y_Predicted_label[i] == label2 and y_validate_label[i] == label1:
                count += 1
        l.append(count)
        count = 0
    confusion_matrix.append(l)
    l = list()
print('Confusion Matrix of model ' + str(NB_Model) + '\n')
print(confusion_matrix)



count, count_correct_match = 0, 0
for label in y_Predicted_label:
    if label == y_validate_label[count]:
        count_correct_match += 1
    count += 1

accuracy = count_correct_match / count

print('\nAccuracy of the Model: ', accuracy)


UnboundLocalError: local variable 'prob_table' referenced before assignment

### Question 4 c

###### Use the model which achieved the best validation accuracy and test it using the test data set. Report a confusion matrix of the results, as well as the test accuracy of the model.

In [ ]:
#Testing the best model with the test data
x_test_data = [line.split(' ')[:-1] for line in test_data]
y_test_label = [line.split(' ')[-1] for line in test_data]
y_Predicted_label = list()
#print(y_test_label)

for vec in x_test_data:
    y_Predicted_label.append(Predicted_Class(vec, Best_Model))
#print(y_Predicted_label)


#computing confusion matrix
class_labels = [1, 2, 3, 4, 5, 6, 7]
confusion_matrix = list()

count = 0

for label1 in class_labels:
    l = list()
    for label2 in class_labels:
        for i in range(len(y_test_label)):
            if y_Predicted_label[i] == label2 and y_test_label[i] == label1:
                count += 1
        l.append(count)
        count = 0
    confusion_matrix.append(l)
    l = list()
print('Confusion Matrix of the Best Model\n')
print(confusion_matrix)



count, count_correct_match = 0, 0
for label in y_Predicted_label:
    if label == y_test_label[count]:
        count_correct_match += 1
    count += 1
    

print('\nAccuracy of the Model: ', count_correct_match / count)